In [2]:
import praw
import pandas as pd

In [3]:
reddit = praw.Reddit(client_id='Zv2KxMP5vx0Q6g', client_secret='NoeyEJSkPNA9IRNXtIYiarJOX-_94g', user_agent='WebScraping CVD')

In [4]:
terms = ['heart surgery', 'heart attack', 'ekg', 'cardiac',
         'stent', 'cholesterol', 'stroke', 'chest pain', 
         'Bypass','Sepsis','Pass','Coronary artery','PCI','CVD', 
         'Cardiovascular','Grafts']

women_topics_dict = {"title":[], \
                "comments":[], \
                "score":[], \
                "id":[], \
                "num_comments": [], \
                "created": [], \
                "body":[]}

men_topics_dict = {"title":[], \
                "comments":[], \
                "score":[], \
                "id":[], \
                "num_comments": [], \
                "created": [], \
                "body":[]}

In [5]:
def get_subreddit_posts(sub_name, search_term):
    posts = reddit.subreddit(sub_name).search(search_term)
    return posts

def create_sr_objects(sub_name):
    subreddit = []
    for term in terms:
        subreddit.append(get_subreddit_posts(sub_name, term))
    return subreddit

def populate_dict(submissions, topics_dict):
    for submission in submissions:
        submission.comments.replace_more(limit=None)
        comments = [comment.body for comment in submission.comments.list()]
        topics_dict["title"].append(submission.title)
        topics_dict["comments"].append(comments)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["num_comments"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)

In [6]:
women_sr = create_sr_objects("AskWomen")
men_sr = create_sr_objects("AskMen")

In [8]:
for i in women_sr:
    populate_dict(i, women_topics_dict)

In [16]:
for i in men_sr:
    populate_dict(i, men_topics_dict)

In [13]:
def clean_dataset(data):
    topics_data = pd.DataFrame(data)
    df = topics_data.explode('comments') #currently, comments is a list... lets expand them into rows
    df = df[df['comments'].notna()]
    
    #the praw search function grabs matches to titles, body of og posts, and comments so lets find them
    search = 'heart surgery|heart attack|ekg|cardiac|stent|cholesterol|stroke|chest pain|bypass|sepsis|pass|coronary artery|pci|cvd|cardiovascular|grafts'
    comments_df = df[df["comments"].str.contains(search, case = False)]
    titles_df = df[df["title"].str.contains(search, case = False)]
    body_df = df[df["body"].str.contains(search, case = False)]
    
    comments = comments_df["comments"]
    titles = titles_df["title"].drop_duplicates()
    body = body_df["body"].drop_duplicates()
    
    #put them all together into 1 column
    final = pd.DataFrame(pd.concat([comments, titles, body])).reset_index(drop = True)
    return final

In [14]:
women_data = clean_dataset(women_topics_dict)

In [17]:
men_data = clean_dataset(men_topics_dict)

In [20]:
women_data.to_csv("data/AskWomen_Data.csv")
men_data.to_csv("data/AskMen_Data.csv")